In [1]:
from sklearn.datasets import fetch_openml #fetch_openml

mnist= fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [2]:
X=mnist["data"]
y=mnist["target"]

In [3]:
X.shape, y.shape

((70000, 784), (70000,))

In [4]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [27]:
X_train=X[:10000]

In [28]:
import numpy as np

X_centered = X_train-X_train.mean(axis=0) #axis=0 is vertical whereas axis=1 is horizontal
U, s, Vt= np.linalg.svd(X_centered)
c1=Vt.T[:,0]
c2=Vt.T[:,1]

#Principal Component Matrix V=(c1 c2 ... cn)

In [29]:
#Xd-proj=XWd
W2=Vt.T[:,:2]
X2D=X_centered.dot(W2)

In [30]:
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
X2D=pca.fit_transform(X)

In [31]:
pca.explained_variance_ratio_

array([0.09746116, 0.07155445])

In [32]:
pca=PCA()
pca.fit(X_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum>=0.95)+1

In [33]:
pca=PCA(n_components=0.95)
X_reduced=pca.fit_transform(X_train)

In [34]:
pca=PCA(n_components=154)
X_reducted=pca.fit_transform(X_train)
X_recovered=pca.inverse_transform(X_reduced)

ValueError: shapes (10000,150) and (154,784) not aligned: 150 (dim 1) != 154 (dim 0)

In [35]:
rnd_pca= PCA(n_components=154, svd_solver="randomized")
X_reduced=rnd_pca.fit_transform(X_train)

In [36]:
from sklearn.decomposition import IncrementalPCA

n_batches=100
inc_pca=IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced = inc_pca.transform(X_train)

ValueError: n_components=154 must be less or equal to the batch number of samples 100.

In [37]:
X_mm=np.memmap(filename, dtype="float32", mode="readonly", shape=(m,n))
batch_size =m//n_batches
inc_pca=IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

NameError: name 'filename' is not defined

In [39]:
from sklearn.decomposition import KernelPCA
rbf_pca=KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced=rbf_pca.fit_transform(X)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf=Pipeline([
    ("kpca", KernelPCA(n_componets=2)),
    ("log_reg", LogisticRegression())
])

param_grid=[{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel":["rbf", "sigmoid"]
}]

grid_search=GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X,y)

print(grid_search.best_params_)

In [ ]:
rbf_pca=KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
X_reduced=rbf_pca.fit_transform(X)
X_preimage=rbf_pca.inverse_transform(X_reduced)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
lle=LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced=lle.fit_transform(X)